In [1]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [2]:
coltypes={'ID':str,'TIC':str,'gaiadr3_source_id':str,'epic_id':str,'Score1':'Int64','Flare1':'Int64',
          'Shelf1':'Int64','Evolution1':'Int64','Score2':'Int64','Flare2':'Int64','Shelf2':'Int64','Evolution2':'Int64'}

In [3]:
lctable=pd.read_csv('lcscoresandchecker.csv',dtype=coltypes)

In [4]:
lctable.head(5)

,pop_id,lcname,idtype,ID,secorcamp,LC_author,cadence,fluxamplitude,TIC,gaiadr3_source_id,epic_id,per,per2,group,age_Myr,disco_paper,paper_author,year,Score1,Flare1,Shelf1,Evolution1,Notes1,Score2,Flare2,Shelf2,Evolution2,Notes2,complex_status1,complex_status2
0,0,EPIC-246676629-13-EVEREST-1800,EPIC,246676629,13,EVEREST,1800,0.075449,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,0,0,1,NaN,1,0,0,0,Very hard to tell what's going on,Complex,Complex
1,0,EPIC-246676629-13-K2SFF-1800,EPIC,246676629,13,K2SFF,1800,0.083411,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,0,0,0,EVEREST is actually better for this object,1,0,0,0,Cadence has lined up weirdly with period,Complex,Complex
2,0,TIC-59129133-5-FFI-30min,TIC,59129133,5,FFI,30min,0.041935,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,1,0,1,NaN,1,1,0,1,NaN,Complex,Complex
3,0,TIC-59129133-32-FFI-10min,TIC,59129133,32,FFI,10min,0.053793,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,0,0,1,NaN,2,0,0,0,NaN,Complex,Complex
4,0,TIC-59129133-43-SPOC-120,TIC,59129133,43,SPOC,120,0.053993,59129133,3392549449695395968,246676629,0.6253,0.6332,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,2,1,1,NaN,2,2,0,1,Literature period might be a little off,Complex,Complex


In [5]:
lctable1=lctable[['pop_id','lcname','idtype','ID','secorcamp','LC_author','cadence','fluxamplitude',
                 'TIC','gaiadr3_source_id','epic_id','per','group','age_Myr','disco_paper',
                 'paper_author','year','Score1','Flare1','Shelf1','Evolution1','Notes1','complex_status1']]
lctable2=lctable[['pop_id','lcname','idtype','ID','secorcamp','LC_author','cadence','fluxamplitude',
                 'TIC','gaiadr3_source_id','epic_id','per2','group','age_Myr','disco_paper',
                 'paper_author','year','Score2','Flare2','Shelf2','Evolution2','Notes2','complex_status2']]
lctable1['per_col']='one'
lctable2['per_col']='two'
lctable1.dropna(subset=['Score1'],inplace=True)
lctable2.dropna(subset=['Score2'],inplace=True)
lctable1.rename(columns={'Score1':'Score','Flare1':'Flare','Shelf1':'Shelf',
                         'Evolution1':'Evolution','Notes1':'Notes',
                         'complex_status1':'complex_status'},inplace=True)
lctable2.rename(columns={'per2':'per','Score2':'Score','Flare2':'Flare','Shelf2':'Shelf',
                         'Evolution2':'Evolution','Notes2':'Notes',
                         'complex_status2':'complex_status'},inplace=True)
lctablecomb=pd.concat([lctable1,lctable2],ignore_index=True)

In [6]:
lctablecomb.head(5)

,pop_id,lcname,idtype,ID,secorcamp,LC_author,cadence,fluxamplitude,TIC,gaiadr3_source_id,epic_id,per,group,age_Myr,disco_paper,paper_author,year,Score,Flare,Shelf,Evolution,Notes,complex_status,per_col
0,0,EPIC-246676629-13-EVEREST-1800,EPIC,246676629,13,EVEREST,1800,0.075449,59129133,3392549449695395968,246676629,0.6253,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,0,0,1,NaN,Complex,one
1,0,EPIC-246676629-13-K2SFF-1800,EPIC,246676629,13,K2SFF,1800,0.083411,59129133,3392549449695395968,246676629,0.6253,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,0,0,0,EVEREST is actually better for this object,Complex,one
2,0,TIC-59129133-5-FFI-30min,TIC,59129133,5,FFI,30min,0.041935,59129133,3392549449695395968,246676629,0.6253,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,1,0,1,NaN,Complex,one
3,0,TIC-59129133-32-FFI-10min,TIC,59129133,32,FFI,10min,0.053793,59129133,3392549449695395968,246676629,0.6253,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,0,0,1,NaN,Complex,one
4,0,TIC-59129133-43-SPOC-120,TIC,59129133,43,SPOC,120,0.053993,59129133,3392549449695395968,246676629,0.6253,Taurus,2.0,Stauffer_2018,Stauffer,2018,2,2,1,1,NaN,Complex,one


In [7]:
numshelves=np.sum(lctablecomb.Shelf)
numevol=np.sum(lctablecomb.Evolution)
numflare=np.count_nonzero(lctablecomb.Flare)

In [8]:
numlcs=lctablecomb.shape[0]

In [9]:
print(f'{numflare} out of {numlcs} light curves have flares ({numflare/numlcs*100:.2f}%).')
print(f'{numshelves} out of {numlcs} light curves have shelves ({numshelves/numlcs*100:.2f}%).')
print(f'{numevol} out of {numlcs} light curves show evolution ({numevol/numlcs*100:.2f}%).')

676 out of 1303 light curves have flares (51.88%).
140 out of 1303 light curves have shelves (10.74%).
389 out of 1303 light curves show evolution (29.85%).


In [10]:
numcomplex=np.count_nonzero(lctablecomb[np.logical_or(lctablecomb.Score==1,lctablecomb.Score==2)].Score)

In [11]:
print(f'{numcomplex} out of {numlcs} light curves show possible or very likely complexity ({numcomplex/numlcs*100:.2f}%).')

944 out of 1303 light curves show possible or very likely complexity (72.45%).


In [12]:
pd.crosstab(lctablecomb.Score, lctablecomb.complex_status)

complex_status,Complex,Double Dip,No Harmonic Spikes,Not Complex
Score,,,,
0,165,28,8,16
1,202,34,2,14
2,653,31,1,7
9,128,7,5,2


In [13]:
lcfilt=lctablecomb[lctablecomb.Score!=9]

In [14]:
corrcols=lcfilt[['Score','Flare','Shelf','Evolution']]

In [15]:
corrcols.corr()

,Score,Flare,Shelf,Evolution
Score,1.000000,0.057198,0.221563,0.262469
Flare,0.057198,1.000000,0.144177,0.087057
Shelf,0.221563,0.144177,1.000000,0.471224
Evolution,0.262469,0.087057,0.471224,1.000000


In [16]:
# sns.pairplot(corrcols)
# plt.show()

In [17]:
def complexovertime(scoreseries):
    scorearray=scoreseries.to_numpy()
    scores=np.unique(scorearray)
    if 0 in scores and 1 in scores and 2 in scores:
        return 'All 3 scores'
    elif 1 in scores and 2 in scores:
        return '1 and 2'
    elif 0 in scores and 1 in scores:
        return '0 and 1'
    elif 0 in scores and 2 in scores:
        return '0 and 2'
    elif 2 in scores:
        return '2 only'
    elif 1 in scores:
        return '1 only'
    elif 0 in scores:
        return '0 only'
    elif 9 in scores:
        return 'all bad'
    else:
        return 'error'

In [188]:
grouped=lctablecomb.groupby(['pop_id','per']).agg(scoreagg=('Score',complexovertime),
                                                  scorelist=('Score',scorestr),
                                                  disco_paper=('disco_paper','first'),
                                                  seclist=('secorcamp',scorestr)).reset_index()

In [190]:
grouped.shape

(273, 6)

In [192]:
np.unique(grouped.scoreagg,return_counts=True) #34+9+20=63 out of 273

(array(['0 and 1', '0 and 2', '0 only', '1 and 2', '1 only', '2 only',
        'All 3 scores', 'all bad'], dtype=object),
 array([ 34,   9,  25,  52,  18, 110,  20,   5]))

In [198]:
grouped[np.logical_and(np.logical_or(np.logical_or(grouped.scoreagg=='0 and 1',
                                                                  grouped.scoreagg=='0 and 2'),
                                                      grouped.scoreagg=='All 3 scores'),
                                       ~grouped.pop_id.isin(decadeids))]

,pop_id,per,scoreagg,scorelist,disco_paper,seclist
3,1,3.632400,0 and 1,0 0 0 1 1 1 0,Stauffer_2018,13 13 5 32 43 44 71
36,23,0.483500,0 and 1,0 0 0 0 1 1,Rebull_2022,10 11 37 38 64 65
40,27,1.862000,All 3 scores,1 2 0,Rebull_2022,11 38 64
63,49,0.254800,All 3 scores,0 1 1 2,Rebull_2022,11 37 38 64
75,59,0.269500,0 and 1,1 0 0,Rebull_2022,11 38 65
79,62,0.349000,0 and 1,0 1 0,Stauffer_2021,11 38 64
84,67,0.205000,0 and 1,0 0 1,Stauffer_2021,11 38 65
90,71,0.267100,All 3 scores,9 2 9 2 1 0 1,Rebull_2016,4 4 42 43 44 70 71
92,72,0.304900,0 and 2,2 2 0 0 0 0 0,Rebull_2016,4 4 42 43 44 70 71
93,73,0.277500,0 and 1,9 0 1 1 0 0,Rebull_2016,4 4 43 44 70 71


IDs to look at: 72, 74, 75, 96, 202, 207, 208

In [220]:
grouped[grouped.scorelist.str.replace(' ', '', regex=False).str.contains(r'(?<=[12])(?:9*0+)+9*(?=[12])')]

,pop_id,per,scoreagg,scorelist,disco_paper,seclist
90,71,0.267100,All 3 scores,9 2 9 2 1 0 1,Rebull_2016,4 4 42 43 44 70 71
95,74,0.606000,All 3 scores,2 2 0 0 0 1 9,Rebull_2016,4 4 42 43 44 70 71
96,75,0.663210,0 and 2,2 2 0 0 2,Popinchalk_2023,3 4 30 31 97
107,84,2.491667,All 3 scores,1 1 2 1 0 1 0 1,Bouma_2024,19 43 44 45 59 71 73 86
119,92,0.279583,0 and 1,1 1 0 1,Bouma_2024,43 44 70 71
126,96,0.567500,All 3 scores,2 0 0 0 1 0 0,Bouma_2024,2 3 29 30 69 96 97
137,105,0.453333,All 3 scores,1 1 1 1 0 1 1 1 0 2 2 0 1 2 1 1 2 2 0 1 1 2 2 ...,Zhan_2019,1 2 3 4 5 6 7 8 9 10 11 12 13 27 28 29 30 31 3...
141,108,0.323333,All 3 scores,1 1 1 1 1 1 1 1 1 0 9 2 1,Bouma_2024,8 9 10 35 36 37 61 62 63 64 88 89 90
142,108,0.392917,All 3 scores,2 2 2 1 2 2 2 1 1 0 9 1 0,Bouma_2024,8 9 10 35 36 37 61 62 63 64 88 89 90
144,110,0.354583,All 3 scores,2 2 1 1 2 2 1 1 2 2 2 2 2 1 1 2 1 1 0 1 2 1 2 ...,Bouma_2024,1 2 3 4 5 6 7 8 9 10 11 12 13 27 28 29 30 31 3...


In [20]:
#grouped[grouped.Score=='all bad']

In [21]:
#grouped[grouped.Score=='0 only']

In [61]:
decadeids=np.array([  3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  16,
        17,  18,  20, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140])

In [23]:
#grouped[grouped.pop_id.isin(decadeids)]

In [63]:
stauffrows=lctablecomb[lctablecomb.pop_id.isin(decadeids)]

In [69]:
def scorestr(scoreseries):
    scorearray=scoreseries.to_numpy()
    scorestr=''
    for score in scorearray:
        scorestr+=str(score)+' '
    return scorestr

In [97]:
def staycomplex(scoreseries):
    scores=scoreseries.to_numpy()
    if not (scores[0]==1 or scores[0]==2 or scores[1]==1 or scores[1]==2):
        if (scores[2]==1 or scores[2]==2):
            return 'gains complexity'
        else:
            return 'never complex'
    elif not (scores[2]==1 or scores[2]==2):
        return 'loses complexity'
    return 'keep complexity'

In [99]:
stauffgroup=stauffrows.groupby(['pop_id', 'per']).agg(score_str = ('Score', scorestr),
                                          stay_complex = ('Score', staycomplex),
                                          authororder= ('LC_author',scorestr)).reset_index()


In [186]:
stauffgroup

,pop_id,per,score_str,stay_complex,authororder
0,3,0.5993,9 2 2,keep complexity,EVEREST K2SFF SPOC
1,4,0.7794,9 2 2,keep complexity,EVEREST K2SFF SPOC
2,5,0.6677,9 0 2,gains complexity,EVEREST K2SFF SPOC
3,5,0.9105,9 0 1,gains complexity,EVEREST K2SFF SPOC
4,6,0.5011,0 9 0,never complex,EVEREST K2SFF SPOC
5,6,0.6043,1 9 1,keep complexity,EVEREST K2SFF SPOC
6,7,0.3802,9 2 2,keep complexity,K2SFF EVEREST SPOC
7,7,0.3996,9 2 2,keep complexity,K2SFF EVEREST SPOC
8,8,1.0529,2 2 2,keep complexity,K2SFF EVEREST SPOC
9,9,0.7158,9 0 0,never complex,EVEREST K2SFF FFI


In [103]:
np.unique(stauffgroup.stay_complex,return_counts=True)

(array(['gains complexity', 'keep complexity', 'loses complexity',
        'never complex'], dtype=object),
 array([ 3, 26,  4,  5]))

In [ ]:
def scorestr(shelfseries):
    shelves=shelfseries.to_numpy()
    

In [139]:
grouped2=lctablecomb.groupby(['pop_id'])['Shelf'].sum().reset_index()

In [141]:
grouped2[grouped2.Shelf>0]

,pop_id,Shelf
0,0,2
8,8,1
12,12,1
18,18,1
22,22,1
...,...,...
174,174,1
182,182,1
186,199,1
189,202,1


In [182]:
grouped2[grouped2.Shelf>1].shape

(26, 2)